#Neural networks; generative adversarial networks (GAN)

#Neural Networks

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0
train_images = train_images[..., tf.newaxis]
test_images = test_images[..., tf.newaxis]
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')


11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/5
844/844 [==============================] - 53s 61ms/step - loss: 0.1938 - accuracy: 0.9402 - val_loss: 0.0587 - val_accuracy: 0.9823
Epoch 2/5
844/844 [==============================] - 50s 59ms/step - loss: 0.0515 - accuracy: 0.9839 - val_loss: 0.0370 - val_accuracy: 0.9882
Epoch 3/5
844/844 [==============================] - 50s 59ms/step - loss: 0.0371 - accuracy: 0.9883 - val_loss: 0.0384 - val_accuracy: 0.9882
Epoch 4/5
844/844 [==============================] - 50s 59ms/step - loss: 0.0285 - accuracy: 0.9913 - val_loss: 0.0356 - val_accuracy: 0.9895
Epoch 5/5
313/313 [==============================] - 4s 13ms/step - loss: 0.0295 - accuracy: 0.9895
Test accuracy: 0.9894999861717224


#Generative Adversarial Networks (GAN)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.datasets import load_iris

# Load Iris dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Target

# Normalize the data
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

# Define dimensions
input_dim = X.shape[1]
latent_dim = 4

# Generator model
def build_generator(latent_dim):
    input_layer = Input(shape=(latent_dim,))
    x = Dense(8, activation='relu')(input_layer)
    x = Dense(input_dim, activation='sigmoid')(x)  # Output layer with sigmoid for [0,1] range
    generator = Model(input_layer, x)
    return generator

# Discriminator model
def build_discriminator():
    input_layer = Input(shape=(input_dim,))
    x = Dense(8, activation='relu')(input_layer)
    x = Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid for binary classification
    discriminator = Model(input_layer, x)
    return discriminator

# Build generator and discriminator
generator = build_generator(latent_dim)
discriminator = build_discriminator()

# Compile discriminator
discriminator.compile(optimizer=Adam(lr=0.0002), loss='binary_crossentropy', metrics=['accuracy'])

# Combined GAN model
z = Input(shape=(latent_dim,))
gen_data = generator(z)
discriminator.trainable = False
validity = discriminator(gen_data)

# Combined model (generator and discriminator)
gan = Model(z, validity)
gan.compile(optimizer=Adam(lr=0.0002), loss='binary_crossentropy')

# Training the GAN
def train_gan(X, epochs=10, batch_size=128):
    for epoch in range(epochs):
        # Generate random noise as input to the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # Generate fake samples using the generator
        gen_samples = generator.predict(noise)

        # Select a random batch of real samples from the dataset
        idx = np.random.randint(0, X.shape[0], batch_size)
        real_samples = X[idx]

        # Labels for real and fake samples
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(real_samples, valid)
        d_loss_fake = discriminator.train_on_batch(gen_samples, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator (via the combined model, where the discriminator weights are frozen)
        g_loss = gan.train_on_batch(noise, valid)

        # Print progress
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")

# Train the GAN
train_gan(X)

# Generate synthetic samples using the trained generator
def generate_samples(num_samples=10):
    noise = np.random.normal(0, 1, (num_samples, latent_dim))
    generated_data = generator.predict(noise)
    return generated_data

# Generate and print synthetic samples
synthetic_samples = generate_samples()
print("Generated Samples:")
print(synthetic_samples)

4/4 [==============================] - 0s 4ms/step
Epoch 0, Discriminator Loss: 0.6862086653709412, Generator Loss: 0.7504369020462036
1/1 [==============================] - 0s 119ms/step
Generated Samples:
[[0.5614743  0.8353513  0.4960865  0.7441142 ]
 [0.30850944 0.6382929  0.5933742  0.41476458]
 [0.37725884 0.4496788  0.5111663  0.49373809]
 [0.19139811 0.6664308  0.59817064 0.44317803]
 [0.35356534 0.42259914 0.5292337  0.5387443 ]
 [0.21520674 0.84862846 0.51508075 0.46262136]
 [0.5094228  0.75314677 0.593027   0.68550324]
 [0.3931726  0.4734123  0.43269256 0.3573815 ]
 [0.4117707  0.43282223 0.58018464 0.6179084 ]
 [0.41951227 0.50581723 0.68218184 0.44485173]]
